In [ ]:
import pandas as pd
import re
import jieba
import nltk
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [ ]:
!pip install jieba
!pip install nltk

nltk.download('punkt')

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\osuse\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True


### Part I: Romanian

In [ ]:
#Load data and preview data
file_path = 'Translation Experiment - BLEU_Romanian.csv'
data_R = pd.read_csv(file_path)

data_R.head()

,Prompt,Google Translate - Romanian,Eng - Romanian (Gemini)
0,Suggest the ingredients needed for Roasted Swe...,Legume:\n\n2-3 cartofi dulci medii (in functie...,Legume:\n\n2-3 cartofi dulci medii (în funcție...
1,"Create an ingredient list in English, Chinese,...",Dovlecei de vară și fritjere ricotta cu sos de...,Friggerele de dovlecei cu ricotta și sos pican...
2,I want to go to the farmers market for a two c...,Supă de legume de primăvară\n\n2 linguri ulei ...,Ciorbă de legume de primăvară\n\nIngrediente:\...
3,Tomorrow is Christmas Eve. What ingredients sh...,Prăjirea festivă de curcan cu sos de afine\n\n...,Pui curcan festiv la tiga cu sos de merisoare ...
4,What ingredients do I need for a dish typical ...,fel principal:\n\nTurcia: Aceasta este vedeta ...,Meniul principal:\n\nCurcan: Vedeta mesei! Ale...


In [ ]:
# Define text cleaning and tokenization function
# Remove newline characters, parentheses, colons, and other punctuation marks
def tokenize_text(text):
    cleaned_text = re.sub(r'[\n\r\t():"\[\],.;!?\\-]', ' ', text)
    tokens = word_tokenize(cleaned_text)
    return ' '.join(tokens)

# Select the columns containing the references and hypotheses (text to evaluate)
data_R['Reference-Romanian'] = data_R['Google Translate - Romanian'].apply(tokenize_text) # Have the Google Translate translations as groundtruth
data_R['Hypothesis-Romanian'] = data_R['Eng - Romanian (Gemini)'].apply(tokenize_text) # Have the Gemini translations as the ones being tested

# Preview first few rows
data_R.head()


,Prompt,Google Translate - Romanian,Eng - Romanian (Gemini),Reference-Romanian,Hypothesis-Romanian
0,Suggest the ingredients needed for Roasted Swe...,Legume:\n\n2-3 cartofi dulci medii (in functie...,Legume:\n\n2-3 cartofi dulci medii (în funcție...,Legume 2 3 cartofi dulci medii in functie de m...,Legume 2 3 cartofi dulci medii în funcție de m...
1,"Create an ingredient list in English, Chinese,...",Dovlecei de vară și fritjere ricotta cu sos de...,Friggerele de dovlecei cu ricotta și sos pican...,Dovlecei de vară și fritjere ricotta cu sos de...,Friggerele de dovlecei cu ricotta și sos pican...
2,I want to go to the farmers market for a two c...,Supă de legume de primăvară\n\n2 linguri ulei ...,Ciorbă de legume de primăvară\n\nIngrediente:\...,Supă de legume de primăvară 2 linguri ulei de ...,Ciorbă de legume de primăvară Ingrediente 2 li...
3,Tomorrow is Christmas Eve. What ingredients sh...,Prăjirea festivă de curcan cu sos de afine\n\n...,Pui curcan festiv la tiga cu sos de merisoare ...,Prăjirea festivă de curcan cu sos de afine Ing...,Pui curcan festiv la tiga cu sos de merisoare ...
4,What ingredients do I need for a dish typical ...,fel principal:\n\nTurcia: Aceasta este vedeta ...,Meniul principal:\n\nCurcan: Vedeta mesei! Ale...,fel principal Turcia Aceasta este vedeta emisi...,Meniul principal Curcan Vedeta mesei Alegeți u...


In [ ]:
#Define a function to calculate BLEU score

def calculate_bleu(reference, candidate):
    smoothing = SmoothingFunction().method1
    return sentence_bleu([reference.split()], candidate.split(), weights=(1, 0, 0, 0), smoothing_function=smoothing)

    # Using nltk's sentence_bleu to compute the BLEU score
    # For 1-gram, we set weights to (1, 0, 0, 0)
    # Adding smoothing function to avoid zero scores


# Apply the function to calculate BLEU score of 1 gram
data_R['BLEU_score'] = data_R.apply(lambda x: calculate_bleu(x['Reference-Romanian'], x['Hypothesis-Romanian']), axis=1)

#Check first few rows
data_R.head()

,Prompt,Google Translate - Romanian,Eng - Romanian (Gemini),Reference-Romanian,Hypothesis-Romanian,BLEU_score
0,Suggest the ingredients needed for Roasted Swe...,Legume:\n\n2-3 cartofi dulci medii (in functie...,Legume:\n\n2-3 cartofi dulci medii (în funcție...,Legume 2 3 cartofi dulci medii in functie de m...,Legume 2 3 cartofi dulci medii în funcție de m...,0.590909
1,"Create an ingredient list in English, Chinese,...",Dovlecei de vară și fritjere ricotta cu sos de...,Friggerele de dovlecei cu ricotta și sos pican...,Dovlecei de vară și fritjere ricotta cu sos de...,Friggerele de dovlecei cu ricotta și sos pican...,0.704082
2,I want to go to the farmers market for a two c...,Supă de legume de primăvară\n\n2 linguri ulei ...,Ciorbă de legume de primăvară\n\nIngrediente:\...,Supă de legume de primăvară 2 linguri ulei de ...,Ciorbă de legume de primăvară Ingrediente 2 li...,0.741071
3,Tomorrow is Christmas Eve. What ingredients sh...,Prăjirea festivă de curcan cu sos de afine\n\n...,Pui curcan festiv la tiga cu sos de merisoare ...,Prăjirea festivă de curcan cu sos de afine Ing...,Pui curcan festiv la tiga cu sos de merisoare ...,0.619565
4,What ingredients do I need for a dish typical ...,fel principal:\n\nTurcia: Aceasta este vedeta ...,Meniul principal:\n\nCurcan: Vedeta mesei! Ale...,fel principal Turcia Aceasta este vedeta emisi...,Meniul principal Curcan Vedeta mesei Alegeți u...,0.704610


In [ ]:
def calculate_bleu(reference, candidate):
    smoothing = SmoothingFunction().method1
    return ([reference.split()], candidate.split(), weights=(0, 0, 1, 0), smoothing_function=smoothing)
    # For 3-gram, we set weights to (0, 0, 1, 0)

# Apply the function to calculate BLEU score of 3 gram
data_R['BLEU_score_3'] = data_R.apply(lambda x: calculate_bleu(x['Reference-Romanian'], x['Hypothesis-Romanian']), axis=1)

data_R.head()

,Prompt,Google Translate - Romanian,Eng - Romanian (Gemini),Reference-Romanian,Hypothesis-Romanian,BLEU_score,BLEU_score_3
0,Suggest the ingredients needed for Roasted Swe...,Legume:\n\n2-3 cartofi dulci medii (in functie...,Legume:\n\n2-3 cartofi dulci medii (în funcție...,Legume 2 3 cartofi dulci medii in functie de m...,Legume 2 3 cartofi dulci medii în funcție de m...,0.590909,0.215385
1,"Create an ingredient list in English, Chinese,...",Dovlecei de vară și fritjere ricotta cu sos de...,Friggerele de dovlecei cu ricotta și sos pican...,Dovlecei de vară și fritjere ricotta cu sos de...,Friggerele de dovlecei cu ricotta și sos pican...,0.704082,0.312500
2,I want to go to the farmers market for a two c...,Supă de legume de primăvară\n\n2 linguri ulei ...,Ciorbă de legume de primăvară\n\nIngrediente:\...,Supă de legume de primăvară 2 linguri ulei de ...,Ciorbă de legume de primăvară Ingrediente 2 li...,0.741071,0.400000
3,Tomorrow is Christmas Eve. What ingredients sh...,Prăjirea festivă de curcan cu sos de afine\n\n...,Pui curcan festiv la tiga cu sos de merisoare ...,Prăjirea festivă de curcan cu sos de afine Ing...,Pui curcan festiv la tiga cu sos de merisoare ...,0.619565,0.188889
4,What ingredients do I need for a dish typical ...,fel principal:\n\nTurcia: Aceasta este vedeta ...,Meniul principal:\n\nCurcan: Vedeta mesei! Ale...,fel principal Turcia Aceasta este vedeta emisi...,Meniul principal Curcan Vedeta mesei Alegeți u...,0.704610,0.353552


#### 1-gram of Romanian BLEU Performance

In [ ]:
# Get minimum and maximum scores and their indices
max_bleu_index_R_1 = data_R['BLEU_score'].idxmax()
min_bleu_index_R_1 =  data_R['BLEU_score'].idxmin()

# Extract corresponding sentences and scores
max_text_R_1_score = data_R.loc[max_bleu_index_R_1, 'BLEU_score']
min_text_R_1_score =  data_R.loc[min_bleu_index_R_1, 'BLEU_score']


min_text_R_1_reference =  data_R.loc[min_bleu_index_R_1, 'Reference-Romanian']
min_text_R_1_hypothesis =  data_R.loc[min_bleu_index_R_1, 'Hypothesis-Romanian']

max_text_R_1_reference = data_R.loc[max_bleu_index_R_1, 'Reference-Romanian']
max_text_R_1_hypothesis = data_R.loc[max_bleu_index_R_1, 'Hypothesis-Romanian']


# Calculate average METEOR score
mean_bleu_R_1 =  data_R['BLEU_score'].mean()
median_bleu_R_1 =  data_R['BLEU_score'].median()

# Print results
print(f"Minimum BLEU 1-gram score: {min_text_R_1_score*100:.4f}%")
print(f"  - Reference: {min_text_R_1_reference}")
print(f"  - Hypothesis: {min_text_R_1_hypothesis}")

print(f"\nMaximum BLEU 1-gram score: {max_text_R_1_score*100:.4f}%")
print(f"  - Reference: {max_text_R_1_reference}")
print(f"  - Hypothesis: {max_text_R_1_hypothesis}")

print(f"\nThe Average BLEU 1-gram of translating Romanian is: {mean_bleu_R_1*100:.3f}%")
print(f"The Median BLEU 1-gram score is: {median_bleu_R_1*100:.3f}%")

Minimum BLEU 1-gram score: 55.9322%
  - Reference: Supa de pui cu taitei Bulion Puteți folosi bulion de pui sau bulion de legume cumpărat din magazin sau puteți să vă faceți singur pentru o supă mai aromată Pui pieptul sau pulpele de pui fiert și mărunțit funcționează bine Dacă nu aveți pui gătit puteți bracona niște piept sau pulpe de pui în bulion în timp ce faceți supa Legume Morcovi tocati Telina taiata cubulete Ceapa tocata optional Taitei Taiteii cu ou sunt clasici dar puteti folosi si alte tipuri de taitei cum ar fi taiteii de orez sau pastina Condimente sare piper ierburi uscate precum cimbru sau oregano Ingrediente optionale usturoi proaspăt tocat Ghimbir proaspăt răzuit frunza de dafin Pătrunjel proaspăt tocat pentru garnitură
  - Hypothesis: Supă de Pui cu Tăiței Supă Poți folosi supă de pui sau supă de legume cumpărată din magazin sau poți să ți prepari propria supă pentru o aromă mai consistentă Pui Puiul fiert și sfărâmat fie din piept fie din pulpe merge bine Dacă nu ai 

#### 3-gram of Romanian BLEU Performance

In [ ]:
# Get minimum and maximum scores and their indices
max_bleu_index_R_3 = data_R['BLEU_score_3'].idxmax()
min_bleu_index_R_3 =  data_R['BLEU_score_3'].idxmin()

# Extract corresponding sentences and scores
max_text_R_3_score = data_R.loc[max_bleu_index_R_3, 'BLEU_score_3']
min_text_R_3_score =  data_R.loc[min_bleu_index_R_3, 'BLEU_score_3']


min_text_R_3_reference =  data_R.loc[min_bleu_index_R_3, 'Reference-Romanian']
min_text_R_3_hypothesis =  data_R.loc[min_bleu_index_R_3, 'Hypothesis-Romanian']

max_text_R_3_reference = data_R.loc[max_bleu_index_R_3, 'Reference-Romanian']
max_text_R_3_hypothesis = data_R.loc[max_bleu_index_R_3, 'Hypothesis-Romanian']


# Calculate average METEOR score
mean_bleu_R_3 =  data_R['BLEU_score_3'].mean()
median_bleu_R_3 =  data_R['BLEU_score_3'].median()

# Print results
print(f"Minimum BLEU 3-gram score: {min_text_R_3_score*100:.4f}%")
print(f"  - Reference: {min_text_R_3_reference}")
print(f"  - Hypothesis: {min_text_R_3_hypothesis}")

print(f"\nMaximum BLEU 3-gram score: {max_text_R_3_score*100:.4f}%")
print(f"  - Reference: {max_text_R_3_reference}")
print(f"  - Hypothesis: {max_text_R_3_hypothesis}")

print(f"\nThe Average BLEU 3-gram of translating Romanian is: {mean_bleu_R_3*100:.3f}%")
print(f"The Median BLEU 3-gram score is: {median_bleu_R_3*100:.3f}%")

Minimum BLEU 3-gram score: 17.5691%
  - Reference: Hummus și batoane de legume Pentru hummus Naut 1 conserve 15 oz de naut scurs si clatit Tahini 2 linguri tahini pasta de seminte de susan Ulei de măsline 2 linguri ulei de măsline Suc de lamaie ¼ cana suc proaspat de lamaie sau dupa gust Usturoi 1 catel de usturoi tocat Condimente ½ linguriță de chimen măcinat Sare si piper negru proaspat macinat dupa gust Pentru batoanele de legume Morcovi pentru copii Bete de telina Felii de castravete Fâșii de ardei gras roșu galben portocaliu sau verde Mazăre cu zahăr Roșii cherry
  - Hypothesis: Hummus și Batoane de Legume Pentru Hummus Năut 1 conservă 425g de năut scurs și clătit Tahini 2 linguri de tahini pastă de susan Ulei de Măsline 2 linguri de ulei de măsline Suc de Lămâie ¼ cană suc de lămâie proaspăt sau după gust Usturoi 1 cățel de usturoi zdrobit Condimente ½ linguriță chimion măcinat Sare și piper negru proaspăt măcinat după gust Pentru Batoane de Legume Morcovi baby Țeline Felii de ca

### Part lI: Chinese

In [ ]:
#Load data and preview data

file_path = 'Translation Experiment - BLEU_Chinese.csv'
data = pd.read_csv(file_path)

data.head()

,Prompt,Google Translate - Chinese,Eng - Chinese (Gemini)
0,Suggest the ingredients needed for Roasted Swe...,蔬菜：\n\n2-3 個中等大小的紅薯（取決於大小）\n1 杯（約 150 克）新鮮蘑菇（例...,食材\n\n2-3 顆中等大小的地瓜\n1 杯（約 150 克）新鮮蘑菇（例如蠔菇、香菇或舞...
1,"Create an ingredient list in English, Chinese,...",櫛瓜和乳清乾酪油條配辣番茄醬（義大利南部）\n原料：\n\n1 個中等大小的西葫蘆（黃色或綠...,南義大利夏季西葫蘆乳清起士餅配香辣番茄醬\n食材\n\n1 個中等大小的西葫蘆（黃色或綠色）...
2,I want to go to the farmers market for a two c...,春菜湯\n\n2湯匙橄欖油\n1 顆中等大小的洋蔥，切丁\n2 根胡蘿蔔，切丁\n2 根芹菜...,春季蔬菜湯\n\n2 湯匙橄欖油\n1 個中等大小的洋蔥，切丁\n2 根胡蘿蔔，切丁\n2 ...
3,Tomorrow is Christmas Eve. What ingredients sh...,節日火雞佐蔓越莓醬炒（英文）\n\n原料：\n\n1磅碎火雞\n1大匙橄欖油\n1 顆中等大...,節日火雞翻炒配蔓越莓醬\n\n食材\n\n半公斤火雞絞肉\r\n1 湯匙橄欖油\r\n1 個...
4,What ingredients do I need for a dish typical ...,主菜：\n\n土耳其：這是節目的明星！ 為您的客人選擇大小合適的整隻火雞。\n餡料/調味品：...,主菜:\n\n火雞 : 這感恩節大餐的主角！ 選擇一個適合用餐人數大小的整隻火雞。\n填充物...


In [ ]:
# Define text cleaning and tokenization function
# Remove newline characters, parentheses, colons, and other punctuation marks
def clean_and_tokenize(text):
    text = re.sub(r'\n|\r', ' ', text)
    text = re.sub(r'[\(\)（）:：]', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    tokens = jieba.cut(text)
    cleaned_text = ' '.join(token for token in tokens if token.strip())
    return cleaned_text

# Select the columns containing the references and hypotheses (text to evaluate)
data['Reference-Chinese'] = data['Google Translate - Chinese'].apply(clean_and_tokenize)
data['Hypothesis-Chinese'] = data['Eng - Chinese (Gemini)'].apply(clean_and_tokenize)

# Preview first few rows
data.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\osuse\AppData\Local\Temp\jieba.cache
Loading model cost 1.857 seconds.
Prefix dict has been built successfully.


,Prompt,Google Translate - Chinese,Eng - Chinese (Gemini),Reference-Chinese,Hypothesis-Chinese
0,Suggest the ingredients needed for Roasted Swe...,蔬菜：\n\n2-3 個中等大小的紅薯（取決於大小）\n1 杯（約 150 克）新鮮蘑菇（例...,食材\n\n2-3 顆中等大小的地瓜\n1 杯（約 150 克）新鮮蘑菇（例如蠔菇、香菇或舞...,蔬菜 23 個 中等 大小 的 紅薯 取決 於 大小 1 杯 約 150 克新鮮 蘑菇 例如...,食材 23 顆 中等 大小 的 地瓜 1 杯 約 150 克新鮮 蘑菇 例如 蠔菇 香菇 或...
1,"Create an ingredient list in English, Chinese,...",櫛瓜和乳清乾酪油條配辣番茄醬（義大利南部）\n原料：\n\n1 個中等大小的西葫蘆（黃色或綠...,南義大利夏季西葫蘆乳清起士餅配香辣番茄醬\n食材\n\n1 個中等大小的西葫蘆（黃色或綠色）...,櫛瓜 和 乳清 乾酪 油條 配辣 番茄 醬義 大利 南部 原料 1 個 中等 大小 的 西葫...,南義 大利 夏季 西葫蘆 乳清 起士 餅配 香辣 番茄 醬 食材 1 個 中等 大小 的 西...
2,I want to go to the farmers market for a two c...,春菜湯\n\n2湯匙橄欖油\n1 顆中等大小的洋蔥，切丁\n2 根胡蘿蔔，切丁\n2 根芹菜...,春季蔬菜湯\n\n2 湯匙橄欖油\n1 個中等大小的洋蔥，切丁\n2 根胡蘿蔔，切丁\n2 ...,春菜湯 2 湯匙 橄欖油 1 顆 中等 大小 的 洋蔥切 丁 2 根胡蘿 蔔 切丁 2 根 ...,春季 蔬菜 湯 2 湯匙 橄欖油 1 個 中等 大小 的 洋蔥切 丁 2 根胡蘿 蔔 切丁 ...
3,Tomorrow is Christmas Eve. What ingredients sh...,節日火雞佐蔓越莓醬炒（英文）\n\n原料：\n\n1磅碎火雞\n1大匙橄欖油\n1 顆中等大...,節日火雞翻炒配蔓越莓醬\n\n食材\n\n半公斤火雞絞肉\r\n1 湯匙橄欖油\r\n1 個...,節日 火雞佐蔓 越莓 醬 炒 英文 原料 1 磅 碎 火雞 1 大匙 橄欖油 1 顆 中等 ...,節日 火雞 翻炒 配蔓 越莓 醬 食材 半公斤 火雞 絞肉 1 湯匙 橄欖油 1 個 中等 ...
4,What ingredients do I need for a dish typical ...,主菜：\n\n土耳其：這是節目的明星！ 為您的客人選擇大小合適的整隻火雞。\n餡料/調味品：...,主菜:\n\n火雞 : 這感恩節大餐的主角！ 選擇一個適合用餐人數大小的整隻火雞。\n填充物...,主菜 土耳其 這是節 目的 明星 為 您 的 客人 選擇 大小 合適 的 整 隻 火雞 餡料...,主菜 火雞 這 感恩 節 大餐 的 主角 選擇 一個 適合 用餐 人數 大小 的 整 隻 火...


In [ ]:
#Define a function to calculate 1-gram BLEU score
def calculate_bleu(reference, candidate):
    smoothing = SmoothingFunction().method1
    return sentence_bleu([reference.split()], candidate.split(), weights=(1, 0, 0, 0), smoothing_function=smoothing)
    # Using nltk's sentence_bleu to compute the BLEU score
    # For 1-gram, we set weights to (1, 0, 0, 0)
    # Adding smoothing function to avoid zero scores

# Apply the function to calculate BLEU score of 1 gram
data['BLEU_score'] = data.apply(lambda x: calculate_bleu(x['Reference-Chinese'], x['Hypothesis-Chinese']), axis=1)

#Check first few rows
data.head()

,Prompt,Google Translate - Chinese,Eng - Chinese (Gemini),Reference-Chinese,Hypothesis-Chinese,BLEU_score
0,Suggest the ingredients needed for Roasted Swe...,蔬菜：\n\n2-3 個中等大小的紅薯（取決於大小）\n1 杯（約 150 克）新鮮蘑菇（例...,食材\n\n2-3 顆中等大小的地瓜\n1 杯（約 150 克）新鮮蘑菇（例如蠔菇、香菇或舞...,蔬菜 23 個 中等 大小 的 紅薯 取決 於 大小 1 杯 約 150 克新鮮 蘑菇 例如...,食材 23 顆 中等 大小 的 地瓜 1 杯 約 150 克新鮮 蘑菇 例如 蠔菇 香菇 或...,0.745606
1,"Create an ingredient list in English, Chinese,...",櫛瓜和乳清乾酪油條配辣番茄醬（義大利南部）\n原料：\n\n1 個中等大小的西葫蘆（黃色或綠...,南義大利夏季西葫蘆乳清起士餅配香辣番茄醬\n食材\n\n1 個中等大小的西葫蘆（黃色或綠色）...,櫛瓜 和 乳清 乾酪 油條 配辣 番茄 醬義 大利 南部 原料 1 個 中等 大小 的 西葫...,南義 大利 夏季 西葫蘆 乳清 起士 餅配 香辣 番茄 醬 食材 1 個 中等 大小 的 西...,0.633333
2,I want to go to the farmers market for a two c...,春菜湯\n\n2湯匙橄欖油\n1 顆中等大小的洋蔥，切丁\n2 根胡蘿蔔，切丁\n2 根芹菜...,春季蔬菜湯\n\n2 湯匙橄欖油\n1 個中等大小的洋蔥，切丁\n2 根胡蘿蔔，切丁\n2 ...,春菜湯 2 湯匙 橄欖油 1 顆 中等 大小 的 洋蔥切 丁 2 根胡蘿 蔔 切丁 2 根 ...,春季 蔬菜 湯 2 湯匙 橄欖油 1 個 中等 大小 的 洋蔥切 丁 2 根胡蘿 蔔 切丁 ...,0.763441
3,Tomorrow is Christmas Eve. What ingredients sh...,節日火雞佐蔓越莓醬炒（英文）\n\n原料：\n\n1磅碎火雞\n1大匙橄欖油\n1 顆中等大...,節日火雞翻炒配蔓越莓醬\n\n食材\n\n半公斤火雞絞肉\r\n1 湯匙橄欖油\r\n1 個...,節日 火雞佐蔓 越莓 醬 炒 英文 原料 1 磅 碎 火雞 1 大匙 橄欖油 1 顆 中等 ...,節日 火雞 翻炒 配蔓 越莓 醬 食材 半公斤 火雞 絞肉 1 湯匙 橄欖油 1 個 中等 ...,0.762500
4,What ingredients do I need for a dish typical ...,主菜：\n\n土耳其：這是節目的明星！ 為您的客人選擇大小合適的整隻火雞。\n餡料/調味品：...,主菜:\n\n火雞 : 這感恩節大餐的主角！ 選擇一個適合用餐人數大小的整隻火雞。\n填充物...,主菜 土耳其 這是節 目的 明星 為 您 的 客人 選擇 大小 合適 的 整 隻 火雞 餡料...,主菜 火雞 這 感恩 節 大餐 的 主角 選擇 一個 適合 用餐 人數 大小 的 整 隻 火...,0.714286


In [ ]:
def calculate_bleu(reference, candidate):
    smoothing = SmoothingFunction().method1
    return sentence_bleu([reference.split()], candidate.split(), weights=(0, 0, 1, 0), smoothing_function=smoothing)
    # For 3-gram, we set weights to (0, 0, 1, 0)

# Apply the function to calculate BLEU score of 3 gram
data['BLEU_score_3'] = data.apply(lambda x: calculate_bleu(x['Reference-Chinese'], x['Hypothesis-Chinese']), axis=1)

data.head()

,Prompt,Google Translate - Chinese,Eng - Chinese (Gemini),Reference-Chinese,Hypothesis-Chinese,BLEU_score,BLEU_score_3
0,Suggest the ingredients needed for Roasted Swe...,蔬菜：\n\n2-3 個中等大小的紅薯（取決於大小）\n1 杯（約 150 克）新鮮蘑菇（例...,食材\n\n2-3 顆中等大小的地瓜\n1 杯（約 150 克）新鮮蘑菇（例如蠔菇、香菇或舞...,蔬菜 23 個 中等 大小 的 紅薯 取決 於 大小 1 杯 約 150 克新鮮 蘑菇 例如...,食材 23 顆 中等 大小 的 地瓜 1 杯 約 150 克新鮮 蘑菇 例如 蠔菇 香菇 或...,0.745606,0.421410
1,"Create an ingredient list in English, Chinese,...",櫛瓜和乳清乾酪油條配辣番茄醬（義大利南部）\n原料：\n\n1 個中等大小的西葫蘆（黃色或綠...,南義大利夏季西葫蘆乳清起士餅配香辣番茄醬\n食材\n\n1 個中等大小的西葫蘆（黃色或綠色）...,櫛瓜 和 乳清 乾酪 油條 配辣 番茄 醬義 大利 南部 原料 1 個 中等 大小 的 西葫...,南義 大利 夏季 西葫蘆 乳清 起士 餅配 香辣 番茄 醬 食材 1 個 中等 大小 的 西...,0.633333,0.284091
2,I want to go to the farmers market for a two c...,春菜湯\n\n2湯匙橄欖油\n1 顆中等大小的洋蔥，切丁\n2 根胡蘿蔔，切丁\n2 根芹菜...,春季蔬菜湯\n\n2 湯匙橄欖油\n1 個中等大小的洋蔥，切丁\n2 根胡蘿蔔，切丁\n2 ...,春菜湯 2 湯匙 橄欖油 1 顆 中等 大小 的 洋蔥切 丁 2 根胡蘿 蔔 切丁 2 根 ...,春季 蔬菜 湯 2 湯匙 橄欖油 1 個 中等 大小 的 洋蔥切 丁 2 根胡蘿 蔔 切丁 ...,0.763441,0.505495
3,Tomorrow is Christmas Eve. What ingredients sh...,節日火雞佐蔓越莓醬炒（英文）\n\n原料：\n\n1磅碎火雞\n1大匙橄欖油\n1 顆中等大...,節日火雞翻炒配蔓越莓醬\n\n食材\n\n半公斤火雞絞肉\r\n1 湯匙橄欖油\r\n1 個...,節日 火雞佐蔓 越莓 醬 炒 英文 原料 1 磅 碎 火雞 1 大匙 橄欖油 1 顆 中等 ...,節日 火雞 翻炒 配蔓 越莓 醬 食材 半公斤 火雞 絞肉 1 湯匙 橄欖油 1 個 中等 ...,0.762500,0.384615
4,What ingredients do I need for a dish typical ...,主菜：\n\n土耳其：這是節目的明星！ 為您的客人選擇大小合適的整隻火雞。\n餡料/調味品：...,主菜:\n\n火雞 : 這感恩節大餐的主角！ 選擇一個適合用餐人數大小的整隻火雞。\n填充物...,主菜 土耳其 這是節 目的 明星 為 您 的 客人 選擇 大小 合適 的 整 隻 火雞 餡料...,主菜 火雞 這 感恩 節 大餐 的 主角 選擇 一個 適合 用餐 人數 大小 的 整 隻 火...,0.714286,0.400000


#### 1-gram of Chinese BLEU Performance

In [ ]:
# Get minimum and maximum scores and their indices
max_bleu_index_C_1 = data['BLEU_score'].idxmax()
min_bleu_index_C_1 =  data['BLEU_score'].idxmin()

# Extract corresponding sentences and scores
max_text_C_1_score = data.loc[max_bleu_index_C_1, 'BLEU_score']
min_text_C_1_score =  data.loc[min_bleu_index_C_1, 'BLEU_score']


min_text_C_1_reference =  data.loc[min_bleu_index_C_1, 'Reference-Chinese']
min_text_C_1_hypothesis =  data.loc[min_bleu_index_C_1, 'Hypothesis-Chinese']

max_text_C_1_reference = data.loc[max_bleu_index_C_1, 'Reference-Chinese']
max_text_C_1_hypothesis = data.loc[max_bleu_index_C_1, 'Hypothesis-Chinese']


# Calculate average METEOR score
mean_bleu_C_1 =  data['BLEU_score'].mean()
median_bleu_C_1 =  data['BLEU_score'].median()

# Print results
print(f"Minimum BLEU 1-gram score: {min_text_C_1_score*100:.4f}%")
print(f"  - Reference: {min_text_C_1_reference}")
print(f"  - Hypothesis: {min_text_C_1_hypothesis}")

print(f"\nMaximum BLEU 1-gram score: {max_text_C_1_score*100:.4f}%")
print(f"  - Reference: {max_text_C_1_reference}")
print(f"  - Hypothesis: {max_text_C_1_hypothesis}")

print(f"\nThe Average BLEU 1-gram of translating Romanian is: {mean_bleu_C_1*100:.3f}%")
print(f"The Median BLEU 1-gram score is: {median_bleu_C_1*100:.3f}%")

Minimum BLEU 1-gram score: 51.8284%
  - Reference: 鱼炸 玉米饼 对于 鱼 白 鱼片 鳕鱼 罗非鱼 鲯 鱼 都 不错 塔科 调味料 橄榄油 酸 橙汁 腌料 用 对于 玉米饼 玉米饼 可 选用 于 煎 玉米饼 的 植物油 使 它们 酥脆 对于 配料 卷心菜 切丝 香菜 切碎 科蒂 亚 奶酪 鳄梨 切片 萨尔 萨舞 或 加洛酱 墨西哥 咖啡 脂可选 青柠角 玛格丽塔 酒 龙舌兰 酒 青柠汁 橙味 利口酒 如君度 或 三秒 甜酒 简单 糖浆 糖 溶于 水 冰 青柠角 装饰 用 可 选 玻璃杯 边缘 的 盐
  - Hypothesis: 鱼炸 玉米饼 鱼肉 白色 鱼片 鳕鱼 罗非鱼 dourado 都 可以 玉米饼 调料 橄榄油 酸 橙汁 用于 腌制 玉米饼 玉米粉 饼 可 选 煎 玉米饼 的 植物油 使 其 酥脆 浇头 卷心菜 切碎 香菜 切碎 Cotija 奶酪 鳄梨 切片 莎莎酱 或 皮 코 德加 洛酱 墨西哥 酸奶 油可选 酸橙角 玛格丽塔 酒 龙舌兰 酒 酸 橙汁 橙味 利口酒 如 君度 或 三重 秒 糖浆 糖 水溶液 冰 酸橙角 用于 装饰 可 选 杯缘 盐

Maximum BLEU 1-gram score: 98.5816%
  - Reference: 印度 綠 咖哩 綠色 調味醬 1 把 新鮮 香菜 葉 ½ 杯 新鮮 薄荷 葉 12 個青 辣椒 依 香料 偏好 調整 1 個 小洋 蔥 切碎 1 英寸 生 薑 去皮 切碎 3 瓣 大蒜 切碎 1 湯匙 綠 荳 蔻 莢 或 ½ 茶匙 磨 荳 蔻 1 茶匙 小茴香 籽 1 茶匙 芫荽 籽 ½ 茶匙 薑 黃 粉 咖哩 部分 1 湯匙 植物油 1 片 月桂 葉可選 1 茶匙 小茴香 籽 1 個 中等 大小 的 洋蔥 切片 400 克去 骨 去皮 雞 胸肉 或 大腿肉 切成 一口 大小 的 塊 狀 也 可以 用 豆腐 代替 適合 素食者 1 罐 400 毫升 椰奶 1 杯 蔬菜 湯 1 湯匙 番茄 醬 或 稱 番茄 沙司 1 湯匙 咖哩粉 可選 鹽 適量 新鮮 香菜 可選用 於 裝飾 煮熟 的 米 飯 或 烤餅 用 於 搭配
  - Hypothesis: 印度 綠 咖哩 綠色 調味醬 1 把 新鮮 香菜 葉 ½ 杯 新鮮 薄荷 葉 12 個青 辣椒 依 香料 偏

#### 3-gram of Chinese BLEU Performance

In [ ]:
# Get minimum and maximum scores and their indices
max_bleu_index_C_3 = data['BLEU_score_3'].idxmax()
min_bleu_index_C_3 =  data['BLEU_score_3'].idxmin()

# Extract corresponding sentences and scores
max_text_C_3_score = data.loc[max_bleu_index_C_3, 'BLEU_score_3']
min_text_C_3_score =  data.loc[min_bleu_index_C_3, 'BLEU_score_3']


min_text_C_3_reference =  data.loc[min_bleu_index_C_3, 'Reference-Chinese']
min_text_C_3_hypothesis =  data.loc[min_bleu_index_C_3, 'Hypothesis-Chinese']

max_text_C_3_reference = data.loc[max_bleu_index_C_3, 'Reference-Chinese']
max_text_C_3_hypothesis = data.loc[max_bleu_index_C_3, 'Hypothesis-Chinese']


# Calculate average METEOR score
mean_bleu_C_3 =  data['BLEU_score_3'].mean()
median_bleu_C_3 =  data['BLEU_score_3'].median()

# Print results
print(f"Minimum BLEU 3-gram score: {min_text_C_3_score*100:.4f}%")
print(f"  - Reference: {min_text_C_3_reference}")
print(f"  - Hypothesis: {min_text_C_3_hypothesis}")

print(f"\nMaximum BLEU 3-gram score: {max_text_C_3_score*100:.4f}%")
print(f"  - Reference: {max_text_C_3_reference}")
print(f"  - Hypothesis: {max_text_C_3_hypothesis}")

print(f"\nThe Average BLEU 3-gram of translating Romanian is: {mean_bleu_C_3*100:.3f}%")
print(f"The Median BLEU 3-gram score is: {median_bleu_C_3*100:.3f}%")

Minimum BLEU 3-gram score: 10.6619%
  - Reference: 鱼炸 玉米饼 对于 鱼 白 鱼片 鳕鱼 罗非鱼 鲯 鱼 都 不错 塔科 调味料 橄榄油 酸 橙汁 腌料 用 对于 玉米饼 玉米饼 可 选用 于 煎 玉米饼 的 植物油 使 它们 酥脆 对于 配料 卷心菜 切丝 香菜 切碎 科蒂 亚 奶酪 鳄梨 切片 萨尔 萨舞 或 加洛酱 墨西哥 咖啡 脂可选 青柠角 玛格丽塔 酒 龙舌兰 酒 青柠汁 橙味 利口酒 如君度 或 三秒 甜酒 简单 糖浆 糖 溶于 水 冰 青柠角 装饰 用 可 选 玻璃杯 边缘 的 盐
  - Hypothesis: 鱼炸 玉米饼 鱼肉 白色 鱼片 鳕鱼 罗非鱼 dourado 都 可以 玉米饼 调料 橄榄油 酸 橙汁 用于 腌制 玉米饼 玉米粉 饼 可 选 煎 玉米饼 的 植物油 使 其 酥脆 浇头 卷心菜 切碎 香菜 切碎 Cotija 奶酪 鳄梨 切片 莎莎酱 或 皮 코 德加 洛酱 墨西哥 酸奶 油可选 酸橙角 玛格丽塔 酒 龙舌兰 酒 酸 橙汁 橙味 利口酒 如 君度 或 三重 秒 糖浆 糖 水溶液 冰 酸橙角 用于 装饰 可 选 杯缘 盐

Maximum BLEU 3-gram score: 95.6835%
  - Reference: 印度 綠 咖哩 綠色 調味醬 1 把 新鮮 香菜 葉 ½ 杯 新鮮 薄荷 葉 12 個青 辣椒 依 香料 偏好 調整 1 個 小洋 蔥 切碎 1 英寸 生 薑 去皮 切碎 3 瓣 大蒜 切碎 1 湯匙 綠 荳 蔻 莢 或 ½ 茶匙 磨 荳 蔻 1 茶匙 小茴香 籽 1 茶匙 芫荽 籽 ½ 茶匙 薑 黃 粉 咖哩 部分 1 湯匙 植物油 1 片 月桂 葉可選 1 茶匙 小茴香 籽 1 個 中等 大小 的 洋蔥 切片 400 克去 骨 去皮 雞 胸肉 或 大腿肉 切成 一口 大小 的 塊 狀 也 可以 用 豆腐 代替 適合 素食者 1 罐 400 毫升 椰奶 1 杯 蔬菜 湯 1 湯匙 番茄 醬 或 稱 番茄 沙司 1 湯匙 咖哩粉 可選 鹽 適量 新鮮 香菜 可選用 於 裝飾 煮熟 的 米 飯 或 烤餅 用 於 搭配
  - Hypothesis: 印度 綠 咖哩 綠色 調味醬 1 把 新鮮 香菜 葉 ½ 杯 新鮮 薄荷 葉 12 個青 辣椒 依 香料 偏

### Part lll: Greek


In [ ]:
#Load data and preview data

file_path = 'Translation Experiment - BLEU_Greek.csv'
data_G = pd.read_csv(file_path)

data_G.head()

,Prompt,Google Translate,Eng - Greek (Gemini)
0,Suggest the ingredients needed for Roasted Swe...,Λαχανικά:\n\n2-3 μέτριες γλυκοπατάτες (ανάλογα...,Λαχανικά:\n\n2-3 μεσαίες γλυκοπατάτες (ανάλογα...
1,"Create an ingredient list in English, Chinese,...",Καλοκαιρινό σκουός και τηγανητές ρικότα με πικ...,Φρίτετες Κολοκυθάκια με Ρικότα και Πικάντικη Σ...
2,I want to go to the farmers market for a two c...,Ανοιξιάτικη σούπα λαχανικών\n\n2 κουταλιές της...,Ανοιξιάτικη Σούπα Λαχανικών \nΥλικά:\n\n2 κουτ...
3,Tomorrow is Christmas Eve. What ingredients sh...,Γιορτινό Stir-Fry γαλοπούλας με σάλτσα Cranber...,Γιορτινό Στιρ-Φράι με Γαλοπούλα και Σάλτσα με ...
4,What ingredients do I need for a dish typical ...,Κύριο πιάτο:\n\nΤουρκία: Αυτή είναι η σταρ του...,Κυρίως Πιάτο \n\nΓαλοπούλα : Αυτή είναι η βασί...


In [ ]:
# Define text cleaning and tokenization function
# Remove newline characters, parentheses, colons, and other punctuation marks

def tokenize_text(text):
    text = str(text)
    cleaned_text = re.sub(r'[\n\r\t():"\[\],.;!?\\-]', ' ', text)
    tokens = word_tokenize(cleaned_text)
    return ' '.join(tokens)

# Select the columns containing the references and hypotheses (text to evaluate)
data_G['Reference-Greek'] = data_G['Google Translate'].apply(tokenize_text)
data_G['Hypothesis-Greek'] = data_G['Eng - Greek (Gemini)'].apply(tokenize_text)

# Preview first few rows
data_G.head()

,Prompt,Google Translate,Eng - Greek (Gemini),Reference-Greek,Hypothesis-Greek
0,Suggest the ingredients needed for Roasted Swe...,Λαχανικά:\n\n2-3 μέτριες γλυκοπατάτες (ανάλογα...,Λαχανικά:\n\n2-3 μεσαίες γλυκοπατάτες (ανάλογα...,Λαχανικά 2 3 μέτριες γλυκοπατάτες ανάλογα με τ...,Λαχανικά 2 3 μεσαίες γλυκοπατάτες ανάλογα με τ...
1,"Create an ingredient list in English, Chinese,...",Καλοκαιρινό σκουός και τηγανητές ρικότα με πικ...,Φρίτετες Κολοκυθάκια με Ρικότα και Πικάντικη Σ...,Καλοκαιρινό σκουός και τηγανητές ρικότα με πικ...,Φρίτετες Κολοκυθάκια με Ρικότα και Πικάντικη Σ...
2,I want to go to the farmers market for a two c...,Ανοιξιάτικη σούπα λαχανικών\n\n2 κουταλιές της...,Ανοιξιάτικη Σούπα Λαχανικών \nΥλικά:\n\n2 κουτ...,Ανοιξιάτικη σούπα λαχανικών 2 κουταλιές της σο...,Ανοιξιάτικη Σούπα Λαχανικών Υλικά 2 κουταλιές ...
3,Tomorrow is Christmas Eve. What ingredients sh...,Γιορτινό Stir-Fry γαλοπούλας με σάλτσα Cranber...,Γιορτινό Στιρ-Φράι με Γαλοπούλα και Σάλτσα με ...,Γιορτινό Stir Fry γαλοπούλας με σάλτσα Cranber...,Γιορτινό Στιρ Φράι με Γαλοπούλα και Σάλτσα με ...
4,What ingredients do I need for a dish typical ...,Κύριο πιάτο:\n\nΤουρκία: Αυτή είναι η σταρ του...,Κυρίως Πιάτο \n\nΓαλοπούλα : Αυτή είναι η βασί...,Κύριο πιάτο Τουρκία Αυτή είναι η σταρ του σόου...,Κυρίως Πιάτο Γαλοπούλα Αυτή είναι η βασίλισσα ...


In [ ]:
#Define a function to calculate BLEU score

def calculate_bleu(reference, candidate):
    smoothing = SmoothingFunction().method1
    return sentence_bleu([reference.split()], candidate.split(), weights=(1, 0, 0, 0), smoothing_function=smoothing)

    # Using nltk's sentence_bleu to compute the BLEU score
    # For 1-gram, we set weights to (1, 0, 0, 0)
    # Adding smoothing function to avoid zero scores

# Apply the function to calculate BLEU score of 1 gram
data_G['BLEU_score'] = data_G.apply(lambda x: calculate_bleu(x['Reference-Greek'], x['Hypothesis-Greek']), axis=1)

#Check first few rows
data_G.head()

,Prompt,Google Translate,Eng - Greek (Gemini),Reference-Greek,Hypothesis-Greek,BLEU_score
0,Suggest the ingredients needed for Roasted Swe...,Λαχανικά:\n\n2-3 μέτριες γλυκοπατάτες (ανάλογα...,Λαχανικά:\n\n2-3 μεσαίες γλυκοπατάτες (ανάλογα...,Λαχανικά 2 3 μέτριες γλυκοπατάτες ανάλογα με τ...,Λαχανικά 2 3 μεσαίες γλυκοπατάτες ανάλογα με τ...,0.708524
1,"Create an ingredient list in English, Chinese,...",Καλοκαιρινό σκουός και τηγανητές ρικότα με πικ...,Φρίτετες Κολοκυθάκια με Ρικότα και Πικάντικη Σ...,Καλοκαιρινό σκουός και τηγανητές ρικότα με πικ...,Φρίτετες Κολοκυθάκια με Ρικότα και Πικάντικη Σ...,0.684226
2,I want to go to the farmers market for a two c...,Ανοιξιάτικη σούπα λαχανικών\n\n2 κουταλιές της...,Ανοιξιάτικη Σούπα Λαχανικών \nΥλικά:\n\n2 κουτ...,Ανοιξιάτικη σούπα λαχανικών 2 κουταλιές της σο...,Ανοιξιάτικη Σούπα Λαχανικών Υλικά 2 κουταλιές ...,0.609485
3,Tomorrow is Christmas Eve. What ingredients sh...,Γιορτινό Stir-Fry γαλοπούλας με σάλτσα Cranber...,Γιορτινό Στιρ-Φράι με Γαλοπούλα και Σάλτσα με ...,Γιορτινό Stir Fry γαλοπούλας με σάλτσα Cranber...,Γιορτινό Στιρ Φράι με Γαλοπούλα και Σάλτσα με ...,0.606717
4,What ingredients do I need for a dish typical ...,Κύριο πιάτο:\n\nΤουρκία: Αυτή είναι η σταρ του...,Κυρίως Πιάτο \n\nΓαλοπούλα : Αυτή είναι η βασί...,Κύριο πιάτο Τουρκία Αυτή είναι η σταρ του σόου...,Κυρίως Πιάτο Γαλοπούλα Αυτή είναι η βασίλισσα ...,0.601090


In [ ]:
def calculate_bleu(reference, candidate):
    smoothing = SmoothingFunction().method1
    return sentence_bleu([reference.split()], candidate.split(), weights=(0, 0, 1, 0), smoothing_function=smoothing)
    # For 3-gram, we set weights to (0, 0, 1, 0)

# Apply the function to calculate BLEU score of 3 gram
data_G['BLEU_score_3'] = data_G.apply(lambda x: calculate_bleu(x['Reference-Greek'], x['Hypothesis-Greek']), axis=1)

data_G.head()

,Prompt,Google Translate,Eng - Greek (Gemini),Reference-Greek,Hypothesis-Greek,BLEU_score,BLEU_score_3
0,Suggest the ingredients needed for Roasted Swe...,Λαχανικά:\n\n2-3 μέτριες γλυκοπατάτες (ανάλογα...,Λαχανικά:\n\n2-3 μεσαίες γλυκοπατάτες (ανάλογα...,Λαχανικά 2 3 μέτριες γλυκοπατάτες ανάλογα με τ...,Λαχανικά 2 3 μεσαίες γλυκοπατάτες ανάλογα με τ...,0.708524,0.459373
1,"Create an ingredient list in English, Chinese,...",Καλοκαιρινό σκουός και τηγανητές ρικότα με πικ...,Φρίτετες Κολοκυθάκια με Ρικότα και Πικάντικη Σ...,Καλοκαιρινό σκουός και τηγανητές ρικότα με πικ...,Φρίτετες Κολοκυθάκια με Ρικότα και Πικάντικη Σ...,0.684226,0.459616
2,I want to go to the farmers market for a two c...,Ανοιξιάτικη σούπα λαχανικών\n\n2 κουταλιές της...,Ανοιξιάτικη Σούπα Λαχανικών \nΥλικά:\n\n2 κουτ...,Ανοιξιάτικη σούπα λαχανικών 2 κουταλιές της σο...,Ανοιξιάτικη Σούπα Λαχανικών Υλικά 2 κουταλιές ...,0.609485,0.204083
3,Tomorrow is Christmas Eve. What ingredients sh...,Γιορτινό Stir-Fry γαλοπούλας με σάλτσα Cranber...,Γιορτινό Στιρ-Φράι με Γαλοπούλα και Σάλτσα με ...,Γιορτινό Stir Fry γαλοπούλας με σάλτσα Cranber...,Γιορτινό Στιρ Φράι με Γαλοπούλα και Σάλτσα με ...,0.606717,0.226281
4,What ingredients do I need for a dish typical ...,Κύριο πιάτο:\n\nΤουρκία: Αυτή είναι η σταρ του...,Κυρίως Πιάτο \n\nΓαλοπούλα : Αυτή είναι η βασί...,Κύριο πιάτο Τουρκία Αυτή είναι η σταρ του σόου...,Κυρίως Πιάτο Γαλοπούλα Αυτή είναι η βασίλισσα ...,0.601090,0.197432


#### 1-gram of Greek BLEU Performance

In [ ]:
# Get minimum and maximum scores and their indices
max_bleu_index_G_1 = data_G['BLEU_score'].idxmax()
min_bleu_index_G_1 =  data_G['BLEU_score'].idxmin()

# Extract corresponding sentences and scores
max_text_G_1_score = data_G.loc[max_bleu_index_G_1, 'BLEU_score']
min_text_G_1_score =  data_G.loc[min_bleu_index_G_1, 'BLEU_score']


min_text_G_1_reference =  data_G.loc[min_bleu_index_G_1, 'Reference-Greek']
min_text_G_1_hypothesis =  data_G.loc[min_bleu_index_G_1, 'Hypothesis-Greek']

max_text_G_1_reference = data_G.loc[max_bleu_index_G_1, 'Reference-Greek']
max_text_G_1_hypothesis = data_G.loc[max_bleu_index_G_1, 'Hypothesis-Greek']


# Calculate average METEOR score
mean_bleu_G_1 =  data_G['BLEU_score'].mean()
median_bleu_G_1 =  data_G['BLEU_score'].median()

# Print results
print(f"Minimum BLEU 1-gram score: {min_text_G_1_score*100:.4f}%")
print(f"  - Reference: {min_text_G_1_reference}")
print(f"  - Hypothesis: {min_text_G_1_hypothesis}")

print(f"\nMaximum BLEU 1-gram score: {max_text_G_1_score*100:.4f}%")
print(f"  - Reference: {max_text_G_1_reference}")
print(f"  - Hypothesis: {max_text_G_1_hypothesis}")

print(f"\nThe Average BLEU 1-gram of translating Romanian is: {mean_bleu_G_1*100:.3f}%")
print(f"The Median BLEU 1-gram score is: {median_bleu_G_1*100:.3f}%")


Minimum BLEU 1-gram score: 56.8966%
  - Reference: Miso σάλτσα σόγιας Ramen Για το ζωμό 4 φλιτζάνια ζωμό κοτόπουλου ή λαχανικών 4 φλιτζάνια νερό 1 κουταλιά της σούπας mirin γλυκό κρασί από ρύζι 1 κουταλιά της σούπας σάκε προαιρετικά 2 3 κουταλιές της σούπας πάστα miso λευκή γλυκιά ή κόκκινη ανάλογα με τις προτιμήσεις σας Φύκια Kombu φύκια προαιρετικά για πρόσθετο βάθος γεύσης Katsuobushi νιφάδες παλαμίδας φύκια προαιρετικά για γαρνίρισμα Για το Ramen 2 μερίδες φρέσκα noodles ramen ή αποξηραμένα noodles ramen σύμφωνα με τις οδηγίες της συσκευασίας Κρεμμύδια σε φέτες για γαρνίρισμα Προαιρετικές επικαλύψεις επιλέξτε οποιαδήποτε ή όλα Χοιρινό Chashu κοιλιά χοιρινή κοκκινιστή Αυγό μαλακό μαριναρισμένο ή σκέτο Σεντόνια από φύκια Nori Βλαστάρια μπαμπού menma Καλαμπόκι Βλαστάρια φασολιών Narutomaki φέτες κέικ ψαριού καμαμπόκο
  - Hypothesis: Μίσο Ραμέν με Σάλτσα Σόγιας Για το ζωμό 4 φλιτζάνια ζωμός κοτόπουλου ή λαχανικών 4 φλιτζάνια νερό 1 κουταλιά της σούπας μίριν γλυκό ρυζο κρασί 1 κουταλιά 

#### 3-gram of Greek BLEU Performance

In [ ]:
# Get minimum and maximum scores and their indices
max_bleu_index_G_3 = data_G['BLEU_score_3'].idxmax()
min_bleu_index_G_3 =  data_G['BLEU_score_3'].idxmin()

# Extract corresponding sentences and scores
max_text_G_3_score = data_G.loc[max_bleu_index_G_3, 'BLEU_score_3']
min_text_G_3_score =  data_G.loc[min_bleu_index_G_3, 'BLEU_score_3']


min_text_G_3_reference =  data_G.loc[min_bleu_index_G_3, 'Reference-Greek']
min_text_G_3_hypothesis =  data_G.loc[min_bleu_index_G_3, 'Hypothesis-Greek']

max_text_G_3_reference = data_G.loc[max_bleu_index_G_3, 'Reference-Greek']
max_text_G_3_hypothesis = data_G.loc[max_bleu_index_G_3, 'Hypothesis-Greek']


# Calculate average METEOR score
mean_bleu_G_3 =  data_G['BLEU_score_3'].mean()
median_bleu_G_3 =  data_G['BLEU_score_3'].median()

# Print results
print(f"Minimum BLEU 3-gram score: {min_text_G_3_score*100:.4f}%")
print(f"  - Reference: {min_text_G_3_reference}")
print(f"  - Hypothesis: {min_text_G_3_hypothesis}")

print(f"\nMaximum BLEU 3-gram score: {max_text_G_3_score*100:.4f}%")
print(f"  - Reference: {max_text_G_3_reference}")
print(f"  - Hypothesis: {max_text_G_3_hypothesis}")

print(f"\nThe Average BLEU 3-gram of translating Romanian is: {mean_bleu_G_3*100:.3f}%")
print(f"The Median BLEU 3-gram score is: {median_bleu_G_3*100:.3f}%")

Minimum BLEU 3-gram score: 19.7432%
  - Reference: Κύριο πιάτο Τουρκία Αυτή είναι η σταρ του σόου Επιλέξτε μια ολόκληρη γαλοπούλα στο μέγεθος για τους καλεσμένους σας Γέμιση/Ντρέσινγκ Ένα αλμυρό μείγμα ψωμιού που συνήθως γεμίζεται μέσα στην κοιλότητα της γαλοπούλας Τα συστατικά μπορεί να διαφέρουν αλλά μπορεί να περιλαμβάνουν κύβους ψωμιού βότανα λαχανικά και μερικές φορές λουκάνικο ή στρείδια Σάλτσα Πλούσια σάλτσα που φτιάχνεται από τηγάνι συχνά με πρόσθετο ζωμό και καρυκεύματα Συνοδευτικά Πουρές πατάτας Ο κρεμώδης πουρές πατάτας είναι ένα βασικό προϊόν για την Ημέρα των Ευχαριστιών Σάλτσα Cranberry Μια τάρτα και πικάντικη σάλτσα φτιαγμένη από cranberries ζάχαρη και μπαχαρικά Μπορεί να σερβιριστεί σε κονσέρβα ή σπιτικό Λαχανικά Μια ποικιλία από ψητά λαχανικά είναι δημοφιλή όπως τα πράσινα φασόλια τα λαχανάκια Βρυξελλών τα καρότα οι γλυκοπατάτες και το καλαμπόκι Αλλα Dinner Rolls Μαλακά και αφράτα ρολά για να απορροφάτε το σάλτσα Βούτυρο Απαραίτητο για πουρέ πατάτας λαχανικά και ρολά δ

### Part lV: Comparison

#### 1-gram of BLEU score

In [ ]:
print(f"The Average BLEU of translating Chinese is: {mean_bleu_C_1*100:.3f}%")
print(f"The Median BLEU score is: {median_bleu_C_1*100:.3f}%")
print(f"The Average BLEU of translating Romanian is: {mean_bleu_R_1*100:.3f}%")
print(f"The Median BLEU score is: {median_bleu_R_1*100:.3f}%")
print(f"The Average BLEU of translating Greek is: {mean_bleu_G_1*100:.3f}%")
print(f"The Median BLEU score is: {median_bleu_G_1*100:.3f}%")

The Average BLEU of translating Chinese is: 71.903%
The Median BLEU score is: 72.865%
The Average BLEU of translating Romanian is: 68.741%
The Median BLEU score is: 68.185%
The Average BLEU of translating Greek is: 70.004%
The Median BLEU score is: 70.144%


#### 3-gram of BLEU score

In [ ]:
print(f"The Average BLEU of translating Chinese is: {mean_bleu_C_3*100:.3f}%")
print(f"The Median BLEU score is: {median_bleu_C_3*100:.3f}%")
print(f"The Average BLEU of translating Romanian is: {mean_bleu_R_3*100:.3f}%")
print(f"The Median BLEU score is: {median_bleu_R_3*100:.3f}%")
print(f"The Average BLEU of translating Greek is: {mean_bleu_G_3*100:.3f}%")
print(f"The Median BLEU score is: {median_bleu_G_3*100:.3f}%")

The Average BLEU of translating Chinese is: 39.725%
The Median BLEU score is: 38.839%
The Average BLEU of translating Romanian is: 33.091%
The Median BLEU score is: 32.616%
The Average BLEU of translating Greek is: 39.966%
The Median BLEU score is: 39.946%
